In [1]:
!pip -q install transformers accelerate>=0.26.0
!pip install -q diffusers


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from diffusers import StableDiffusionXLPipeline
from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker
from transformers import CLIPImageProcessor, CLIPTokenizer, CLIPModel
import torch
from torchvision.transforms.functional import pil_to_tensor, to_pil_image
from PIL import Image as PIL_Image

import torch.nn as nn

In [3]:
# Load the model
device="cuda"
dtype=torch.float16

feature_extractor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
safety_checker = StableDiffusionSafetyChecker.from_pretrained(
    "CompVis/stable-diffusion-safety-checker").to(device)

In [ ]:
def add_nsfw_embedding(image: PIL_Image, weight: int):
    safety_checker_input = feature_extractor(image, return_tensors="pt").to(device)

    pooled_output  = safety_checker.vision_model(safety_checker_input.pixel_values.to(dtype))[1]
    image_embeds = safety_checker.visual_projection(pooled_output)
    
    safety_checker.concept_embeds = nn.Parameter(torch.cat([safety_checker.concept_embeds, image_embeds]))
    safety_checker.concept_embeds_weights = nn.Parameter(torch.cat([safety_checker.concept_embeds_weights, torch.Tensor([0.6]).to(device)]))

In [5]:
def do_safety_check(image):
    safety_checker_input = feature_extractor(image, return_tensors="pt").to(device)
    image, has_nsfw_concept = safety_checker(
        images=pil_to_tensor(image).unsqueeze(0), clip_input=safety_checker_input.pixel_values.to(dtype)
        )
    
    return has_nsfw_concept

In [6]:
image = PIL_Image.open("green-fedora-2.png").convert("RGB")
do_safety_check(image)

[False]

In [7]:
# Image we don't want to allow
image_to_ban = PIL_Image.open("green-fedora.png").convert("RGB")
add_nsfw_embedding(image_to_ban, 0.6)

In [8]:
image = PIL_Image.open("green-fedora-2.png").convert("RGB")
do_safety_check(image)

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


[True]

In [9]:
image = PIL_Image.open("cute-cat.jpg").convert("RGB")
do_safety_check(image)

[False]